<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Lic. Ingeniería Financiera </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. María Fernanda Pinedo Talango - if705971@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 0: Herramientas Computacionales </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Primavera 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/'>https://github.com/ferpinedot/LAB_0_MFPT</a></font>

---

# Resumen

Este laboratorio sirve para generar nueva información de los precios a partir de los históricos en formato  OHLC (Open, High, Low, Close). Esta nueva información generada, la mayoría almacenada como nuevas columnas dentro del DataFrame que contiene los precios históricos OHLC, puede ser utilizada también como "variables explicativas" en modelos predictivos (ya sean de regresión o de clasificación). El enfóque de los cálculos que deberás de realizar para este laboratorio es "estadístico" utilizando información de las "velas".  También vas a poner en práctica algunos códigos simples para graficar utilizando la librería de Plotly, todo esto utilizando una lógica de organizar el código con base a "funciones" y scripts separados (algo bastante útil para proyectos aplicados de python).

Deberás de incluir en esta entrega sólamente la liga del repositorio donde se encuentra tu laboratorio. El repositorio deberá de contener, por lo menos, dos archivos. El script de tus códigos hechos (un archivo .py) y el notebook con tu versión final, donde escribirás texto y código con comentarios (utilizando el formato de notebooks que te pase).

# Código

In [159]:
import datetime                                     # Para visulizacion de tiempo
import numpy as np                                  # Para datos numéricos

import funciones as fn                              # Para procesamiento de datos
import visualizaciones as vs                        # Para visualizacion de datos
import pandas as pd                                 # Procesamiento de datos
from datos import OA_Ak                             # Importar token para API de OANDA
import plotly.express as px                         # Para librería de nueva gráfica al último inciso

Descargar precios de OANDA

In [134]:
OA_In = "EUR_USD"                  # Instrumento
OA_Gn = "D"                        # Granularidad de velas
fini = pd.to_datetime("2019-07-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final

Descargar los precios masivos ya con las especificaciones mencionadas arriba, utilizando funciones para la descarga de datos escritas en la sección de funciones.py

In [136]:
df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

Graficar Open High Low Close (OHLC) prices en plotly

In [137]:
vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :])
vs_grafica1.show()


Conteo de velas

In [138]:
# multiplicador de precios, la diferencia entre una vela y otra suele medirse por medio de pips
pip_mult = 10000

# -- 0A.1: Hora
df_pe['Hora'] = [df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp']))]

# -- 0A.2: Dia de la semana.
df_pe['Dia'] = [df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp']))]

# -- 0B: Boxplot de amplitud de velas (close - open).
df_pe['CO'] = (df_pe['Close'] - df_pe['Open'])*pip_mult

# -- ------------------------------------------------------------ Graficar Boxplot plotly -- #
vs_grafica2 = vs.g_boxplot_varios(p0_data=df_pe[['CO']], p1_norm=False)
vs_grafica2.show()


## Tarea

### Funciones a desarrollar


##### (1pt) ['mes'] : Mes en el que ocurrió la vela.
Utilizando la columna de TimeStamp calcula el "Mes" en el que ocurrió la vela.

Este ejercicio fue muy similar al visto en clase, en donde se utilizó compresión de listas para encontrar el mes en el TimeStamp en toda la longitud de la columna TimeStamp

In [139]:
df_pe['Mes'] = [df_pe['TimeStamp'][i].month for i in range(0, len(df_pe['TimeStamp']))]

In [140]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7


##### (1pt) ['sesion'] : Sesión de la vela
Sesión bursátil en la que ocurrió la vela,  el valor dentro de la columna deberás de colocarlo siguiendo la siguiente regla:

* 'asia':  si en la columna ['hora'] tiene alguno de estos valores -> 22, 23, 0, 1, 2, 3, 4, 5, 6, 7
* 'asia_europa': si en la columna ['hora'] tiene alguno de estos valores -> 8
* 'europa': si en la columna ['hora'] tiene alguno de estos valores -> 9, 10, 11, 12 
* 'europa_america': si en la columna ['hora'] tiene alguno de estos valores -> 13, 14, 15, 16
* 'america': si en la columna ['hora'] tiene alguno de estos valores -> 17, 18, 19, 20, 21

Recuerda que el ['TimeStamp'] de los precios que están descargándose en el código está en huso horario UTC, así que no aplicará a las horas de Guadalajara", sino, a las horas de cualquier centro bursátil que esté bajo el huso horario UTC, como lo es el caso de Londres que tiene huso horario GMT = 0.

In [141]:
# Con las horas dadas previamente, se hace una clasificación de en qué Zona Horaria se realizaron los movimientos

def sesion(hora):
    if hora in [22, 23, 0, 1, 2, 3, 4, 5, 6, 7]: 
        TimeZone = 'Asia'
    elif hora in [8]:
        TimeZone = 'Asia_Europa'
    elif hora in [9, 10, 11, 12]:
        TimeZone = 'Europa'
    elif hora in [13, 14, 15, 16]:
        TimeZone = 'Europa_America'
    else:
        TimeZone = 'America'
    return TimeZone

In [142]:
# Para indicar en que uso horario de los mencionados anteriormente, se ubican en este caso los primeros 5
# movimientos de todos los descargados 

df_pe['Sesión de la vela'] = [sesion(int(df_pe['Hora'][i])) for i in range(0, len(df_pe['Hora']))]

In [143]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes,Sesión de la vela
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7,America
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7,America
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7,America
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7,America
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7,America


##### (1pt) ['oc']: Amplitud de vela (en pips).
Calcular la diferencia entre las columnas ['Open'] y ['Close'], expresarla en pips.

Como se había mencionado anteriormente, la diferencia entre velas, normalmente es expresado en pips, los cuales son equivalentes a 10,000 unidades. En el caso a continuación, se mide la diferencia en pips del precio de apertura, con respecto al precio de cierre (una resta).

In [144]:
# Se realiza la resta de las dos columnas por fila y la diferencia resultante por fila, se multiplica por 10,000 unidades 
# (un pip)

df_pe['OC'] = (df_pe['Open'] - df_pe['Close'])*pip_mult

In [145]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes,Sesión de la vela,OC
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7,America,0.9
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7,America,15.5
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7,America,2.9
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7,America,-46.6
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7,America,-3.7


##### (1pt) ['hl']: Amplitud de extremos (en pips).
Calcular la diferencia entre las columnas ['High'] y ['Low'], expresarla en pips.

En el caso de amplitud de extremos, conocidos como las líneas que salen de la misma vela hacia arriba o hacia abajo, se hace una medición, igualmente en pips, de la diferencia que hay entre el precio más alto en el momento, contra el precio más bajo

In [146]:
# Se realiza la resta de las dos columnas por fila y la diferencia resultante por fila se multiplica por 10,000 unidades (un pip)

df_pe['HL'] = (df_pe['High'] - df_pe['Low'])*pip_mult

In [147]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes,Sesión de la vela,OC,HL
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7,America,0.9,3.1
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7,America,15.5,27.3
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7,America,2.9,25.3
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7,America,-46.6,62.3
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7,America,-3.7,40.4


##### (.5pt) ['sentido'] : Sentido de la vela (alcista o bajista)
En esta columna debes de asignarle el valor de 'alcista' para cuando ['Close'] >= ['Open'] y 'bajista' en el caso contrario.


Para saber si el sentido de la vela es alcista o bajista, es necesario saber la diferencia entre el precio de Cierre con el precio de Apertura, si el precio de Cierre es mayor que el precio de Apertura, la vela tiene una tendencia alcista ya que aumentó el valor de la divisa (en este caso), en caso contrario, que el precio de Cierre sea menor al precio de Apertura, se trata de una tendencia bajista, debido a que el valor de la divisa disminuyó.

In [148]:
# Para saber si el precio es positivo o negativo, se hacen las validaciones con un if mencionando que si la diferencia 
# entre el precio de Cierre con el precio de Apertura es mayor o igual a cero, entonces el sentido de la vela sería alcista
# a diferencia contraria de que si el sentido es menor a cero, se trataría de una tendencia bajista


def sentido(CO):
    if CO >= 0:
        sentido = 'alcista'
    else:
        sentido = 'bajista'
    return sentido


# A continuación convertí los resultados de las tendencias alcistas con un valor de 1, y las bajistas con un valor de -1 para
# posteriormente poder sumarlas y contar, haciendo patrones de cada cuántas tendencias alcistas, sigue una bajista y poder 
# tratar de predecir el comportamiento de las velas en esta divisa

def sentido_int(sentido):
    if sentido == 'alcista':
        sentido_int = 1
    else:
        sentido_int = -1
    return sentido_int
        

In [149]:
df_pe['Sentido de la vela'] = [sentido(int(df_pe['CO'][i])) for i in range(0, len(df_pe['CO']))]

In [150]:
df_pe['Sentido en num'] = [sentido_int(str(df_pe['Sentido de la vela'][i])) for i in range(0, len(df_pe['Sentido de la vela']))]

In [151]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes,Sesión de la vela,OC,HL,Sentido de la vela,Sentido en num
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7,America,0.9,3.1,alcista,1
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7,America,15.5,27.3,bajista,-1
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7,America,2.9,25.3,bajista,-1
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7,America,-46.6,62.3,alcista,1
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7,America,-3.7,40.4,alcista,1


##### (.5pt) ['sentido_c'] Conteo de velas consecutivas alcistas/bajistas.

En el DataFrame de los precios OHLC, para cada renglon, ir acumulando el valor de velas consecutivas ALCISTAS o BAJISTAS e ir haciendo el conteo de ocurrencia para cada caso. Se comienza el conteo a partir de la primera repetición, por ejemplo, ['sentido_c'] tendrá un 2  en el tiempo t cuando en el tiempo t-2 y tiempo t-1 haya sido el mismo valor que en el tiempo t. En este ejemplo ['sentido_c'] tendría un 2 (en el tiempo t-2 fue la primera vela, y la vela en tiempo t-1 y en tiempo t fueron 2 velas fueron consecutivamente en el mismo sentido).

En el caso de conteo de velas, definí sentido_c como el conteo del sentido_int que es la columna de los sentidos alcista y bajista determinadas por números para poder así sumarlos, la suma sería como ejemplo el primer valor en este caso de la columna sentido_c en la primera fila sería 0, ya que solo hubo tendencia alcista, de la segunda fila sería igualmente 0 ya que alcista es diferente a bajista, pero en la tercera fila sería un 1 ya que una vez se repitió la tendencia bajista, y en la cuarta fila, sería 0 de nuevo ya que alcista y bajista son diferentes

In [174]:
def conteo_velas(count, alcista, bajista):
        if 'alcista' == 'alcista':
            count += 1
        elif 'bajista' == 'bajista':
            count += 1
        else: 
            count = 0
        return count

In [176]:
df_pe['Sentido_C'] = [conteo_velas(str(df_pe['Sentido en num'][i])) for i in range(0,len(df_pe['Sentido en num']))]

TypeError: conteo_velas() missing 2 required positional arguments: 'alcista' and 'bajista'

In [ ]:
def sentido_c(sentido_int):
    for i in range(0, len(df_pe['Sentido en num'])):
        sentidos = sentido_int[i] + sentido_int[i+1]
        if sentido_int[i] == sentido_int[i+1]:
            sentido_c[i] = np.sum(abs(sentidos))
        elif sentido_int[i] != sentido_int[i+1]:
            sentido_c = 0

In [153]:
#df_pe['sentido_c'] = sum(abs(sentido_int[i])-1 if sentido_int[i] > 0 else 0 for i in range(len(sentido_int)))

In [154]:
#sum( abs(xRatings[i] - yRatings[i]) for i in range(len(xRatings)))

In [155]:
df_pe['sentido_c'] = [sentido_int(int(df_pe['Sentido en num'][i])) for i in range(0, len(df_pe['Sentido en num']))]

In [156]:
df_pe.head()

,TimeStamp,Open,High,Low,Close,Hora,Dia,CO,Mes,Sesión de la vela,OC,HL,Sentido de la vela,Sentido en num,sentido_c
0,2019-07-05 20:00:00+00:00,1.12264,1.12278,1.12247,1.12255,20,4,-0.9,7,America,0.9,3.1,alcista,1,-1
1,2019-07-07 20:00:00+00:00,1.12248,1.12344,1.12071,1.12093,20,6,-15.5,7,America,15.5,27.3,bajista,-1,-1
2,2019-07-08 20:00:00+00:00,1.12095,1.12188,1.11935,1.12066,20,0,-2.9,7,America,2.9,25.3,bajista,-1,-1
3,2019-07-09 20:00:00+00:00,1.12070,1.12643,1.12020,1.12536,20,1,46.6,7,America,-46.6,62.3,alcista,1,-1
4,2019-07-10 20:00:00+00:00,1.12533,1.12858,1.12454,1.12570,20,2,3.7,7,America,-3.7,40.4,alcista,1,-1


##### (1pt) Ventanas móviles de volatilidad
Utiliza la columna de ['hl'] como una medida de "volatilidad" en pips de las velas. Con esta columna, genera las siguientes columnas haciendo una "ventana móvil" del máximo de esos últimos n valores. Las columnas serán 3, una para cada valor de la "volatilidad móvil" para 5, 25 y 50 velas de histórico respectivamente.

* ['volatilidad_5']: Utilizando la información de las 5 anteriores velas.
* ['volatilidad_25']: Utilizando la información de las 25 anteriores velas.
* ['volatilidad_50']: Utilizando la información de las 50 anteriores velas.
Recuerda que la "volatilidad" en una serie de tiempo financiera es, usualmente, la desviación estándar de los rendimientos, sin embargo, uno puedeo proponer otros "estadísticos" para representar la "variabilidad" entre los datos. En este caso, probaremos generar esta información sólo tomando en cuenta la columna ['hl']. Así que, no es necesario calcular rendimientos en esta ocasión.

Se calcula la volatilidad de la rentabilidad de la divisa que estamos manejando, utilizando ventanas móviles de una determinada amplitud, en este caso sería de 5, 25 y 50 períodos. La ventana se va desplazando a lo largo de la muestra conforme avanza el tiempo y en cada período se calcula la volatilidad de las rentabilidades observadas en el período de tiempo indicados. Así se va construyendo una serie temporal de volatilidades para cada amplitud de ventana.

In [177]:
# Para calcular las ventanas móviles existe una función llamada rolling que va haciendo "móvil" la ventana, moviéndola de lugar
# en el período de tiempo indicado, calculando la desviación estándar/volatilidad entre las velas

volatilidad_5 = df_pe['HL'].rolling(window=5).std()
volatilidad_25 = df_pe['HL'].rolling(window=25).std()
volatilidad_50 = df_pe['HL'].rolling(window=50).std()

volatilidad_5
#volatilidad_25
#volatilidad_50

0            NaN
1            NaN
2            NaN
3            NaN
4      21.738951
         ...    
126    15.743316
127    30.876577
128    31.222860
129    29.801728
130    30.071166
Name: HL, Length: 131, dtype: float64

##### (1pt) Gráfica con Plotly
Realiza una propuesta de gráfica utilizando alguna de las columnas que has generado y la librería plotly. Las reglas son las siguientes:

1. Tiene que tener título de gráfica
2. Tiene que tener título de eje x y etiquetas de eje x
3. Tiene que tener título de eje y y etiquetas de eje y
4. Se debe de poder visualizar una leyenda (en cualquier posición).
 

Realicé una gráfica de pastel donde debe de calcular qué porcentaje de la columna del sentido de las velas es alcista y qué porcentaje es bajista.

In [158]:
fig = px.pie(data_frame=df_pe, values='Sentido de la vela', labels={'alcista', 'bajista'}, title='Sentido de las velas')
fig.show()